In [1]:
import pandas as pd
import numpy as np
import spacy
from nltk import tokenize
import googletrans
from googletrans import Translator
import time
import math
import jieba
import random
import urllib
import requests
import csv
import gzip

In [85]:
df = pd.read_csv('seame_data_annotated.csv')
df.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,row_number,original_row_number,file,type,start_time,end_time,transcript_2014,filtered_transcript_2014,transcript_2015,filtered_transcript_2015,...,Maximum pitch,Minimum intensity,Mean intensity,Maximum intensity,Jitter,Shimmer,HNR,Speaking rate,Std Dev Pitch,Std Dev Intensity
0,2,NaN,01NC01FBX_0101,Conversation,0,6010,NaN,NaN,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,9360.0,01NC02FBY_0101,Conversation,0,1470,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",十月 you mean finals,"['十月', 'you', 'mean', 'finals']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,NaN,01NC02FBY_0101,Conversation,2201,2605,NaN,NaN,我们,['我们'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,NaN,01NC02FBY_0101,Conversation,3037,3804,NaN,NaN,进 [啊] 没有,"['进', '没有']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,NaN,01NC02FBY_0101,Conversation,6675,7583,NaN,NaN,(ppl) 等一下 (ppb),"['等', '一下']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
len(df)

110145

In [87]:
df.columns

Index(['row_number', 'original_row_number', 'file', 'type', 'start_time',
       'end_time', 'transcript_2014', 'filtered_transcript_2014',
       'transcript_2015', 'filtered_transcript_2015', 'length_of_turn',
       'start_of_code_switches', 'length_of_code_switched_segments',
       'direction_of_code_switches', 'number_of_code_switches',
       'intersentential_code_switch?', 'dialog_act', 'type_of_code_switching',
       'language', 'conversation', 'speaker', 'Minimum pitch', 'Mean pitch',
       'Maximum pitch', 'Minimum intensity', 'Mean intensity',
       'Maximum intensity', 'Jitter', 'Shimmer', 'HNR', 'Speaking rate',
       'Std Dev Pitch', 'Std Dev Intensity'],
      dtype='object')

In [88]:
df = df[['file', 'type','transcript_2015', 'filtered_transcript_2015', 'length_of_turn',
       'start_of_code_switches', 'length_of_code_switched_segments',
       'direction_of_code_switches', 'number_of_code_switches',
       'intersentential_code_switch?','type_of_code_switching','language', 'conversation', 'speaker']]
df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2
2,01NC02FBY_0101,Conversation,我们,['我们'],1,-1,-1,-1,0,-1,-1,ZH,1,2
3,01NC02FBY_0101,Conversation,进 [啊] 没有,"['进', '没有']",2,-1,-1,-1,0,-1,-1,ZH,1,2
4,01NC02FBY_0101,Conversation,(ppl) 等一下 (ppb),"['等', '一下']",2,-1,-1,-1,0,-1,-1,ZH,1,2


In [5]:
csw_df = df[df['language']=='CS']
csw_df = csw_df[csw_df['length_of_turn']!=0]
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1
14,01NC02FBY_0101,Conversation,[哦] cause 那个 #cui#,"['cause', '那个', '#cui#']",3,"[1, 2]","[1, 1]","['eng > man', 'man > other']",2,N,I,CS,1,2


In [6]:
len(csw_df)

56906

In [7]:
marking = []
for lst in csw_df['direction_of_code_switches']:
    #print(type(lst))
    as_list = lst.strip('][').split(', ')
    #print(as_list)
    if as_list[0] == "'man > eng'":
        marking.append('Y')
    else:
        marking.append('N')
csw_df['marking'] = marking
csw_df = csw_df[csw_df['marking']=='Y']
len(csw_df)

34117

In [8]:
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y


In [9]:
first_cs_word = []
for index, row in csw_df.iterrows():
    lst = row['start_of_code_switches']
    as_list = lst.strip('][').split(', ')
    idx = int(as_list[0])
    word = row['filtered_transcript_2015'].strip('][').split(', ')[idx]
    if word.startswith("'") and word.endswith("'"):
        word = word[1:-1]
    first_cs_word.append(word)
csw_df['first_cs_word'] = first_cs_word

In [10]:
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top


In [11]:
nlp = spacy.load("en_core_web_sm")

In [12]:
tags = []
for word in list(csw_df['first_cs_word']):
    doc = nlp(str(word))
    for token in doc:
        tags.append(token.tag_)
        break

In [13]:
coded_POS = []
for tag in tags:
    if tag == 'NN' or tag == 'NNP' or tag == 'NNS' or tag == 'NNPS':
        coded_POS.append(0)
    elif 'V' in tag:
        coded_POS.append(1)
    else:
        coded_POS.append(2)

In [14]:
csw_df['first_cs_word_eng_pos'] = coded_POS
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but,2
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you,2
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah,0
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but,2
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top,2


In [15]:
word_length = []
for word in list(csw_df['first_cs_word']):
    word_length.append(len(word))
csw_df['word_length_eng'] = word_length
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but,2,3
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you,2,3
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah,0,3
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but,2,3
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top,2,3


In [15]:
#!pip install googletrans==4.0.0rc1
print(googletrans.LANGUAGES) #check language codes
print(googletrans.__version__) #make sure this is 4.0.0-rc.1
translator = Translator()
translator.raise_Exception = True

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [55]:
en_translated_lines = []
for i in range(len(list(csw_df['filtered_transcript_2015']))):
    if i%100==0:
        print(i)
    ini_list = list(csw_df['filtered_transcript_2015'])[i]
    as_list = ini_list.strip('][').split(', ')
    as_list = [word[1:-1] for word in as_list if word.startswith("'") and word.endswith("'")]
    as_str = ' '.join(as_list)
    
    try:
        result = translator.translate(as_str, src='zh-cn', dest='en').text
        time.sleep(0.3)
        en_translated_lines.append(result.lower())
    except:
        en_translated_lines.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [56]:
en_translated_lines.count('TODO')

169

In [57]:
en_translated_lines.index('TODO')

1146

In [58]:
csw_df['english_translation'] = en_translated_lines
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ..."
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you,2,3,october you mean finals
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah,0,3,can't tell whether we are yah
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that..."
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top,2,3,because top four


In [59]:
csw_df.to_csv('seame_with_eng_translations.csv')

In [3]:
csw_df = pd.read_csv('seame_with_eng_translations.csv')
csw_df.head()

,Unnamed: 0,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation
0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ..."
1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you,2,3,october you mean finals
2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah,0,3,can't tell whether we are yah
3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that..."
4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top,2,3,because top four


In [4]:
len(csw_df)

34117

In [11]:
first_cs_word_eng = list(csw_df['first_cs_word'])
first_cs_word_zh = []
for i in range(len(first_cs_word_eng)):
    word = first_cs_word_eng[i]
    if i%100==0:
        print(i)
    try:
        result = translator.translate(word, src='en', dest='zh-cn').text
        time.sleep(0.3)
        first_cs_word_zh.append(result)
    except:
        first_cs_word_zh.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [12]:
first_cs_word_zh.count('TODO')

6

In [13]:
csw_df['first_cs_word_zh'] = first_cs_word_zh
csw_df.head()

,Unnamed: 0,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,...,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh
0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,...,I,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但
1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,...,I,CS,1,2,Y,you,2,3,october you mean finals,你
2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,...,-1,CS,1,2,Y,yah,0,3,can't tell whether we are yah,雅虎
3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,...,I,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that...",但
4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,...,O,CS,1,1,Y,top,2,3,because top four,顶部


In [22]:
first_cs_word_zh_len = []
for word in first_cs_word_zh:
    first_cs_word_zh_len.append(len(word))
print(max(first_cs_word_zh_len))

22


In [26]:
csw_df['word_length_zh'] = first_cs_word_zh_len
csw_df.head()

,Unnamed: 0,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,...,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh
0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,...,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1
1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,...,CS,1,2,Y,you,2,3,october you mean finals,你,1
2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,...,CS,1,2,Y,yah,0,3,can't tell whether we are yah,雅虎,2
3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,...,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that...",但,1
4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,...,CS,1,1,Y,top,2,3,because top four,顶部,2


In [33]:
csw_df.to_csv('seame_with_eng_translations.csv')

In [4]:
csw_df = pd.read_csv('seame_with_eng_translations.csv')
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,...,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh
0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",...,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1
1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],...,CS,1,2,Y,you,2,3,october you mean finals,你,1
2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],...,CS,1,2,Y,yah,0,3,can't tell whether we are yah,雅虎,2
3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",...,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that...",但,1
4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],...,CS,1,1,Y,top,2,3,because top four,顶部,2


In [149]:
nlp = spacy.load("zh_core_web_sm")

In [8]:
tags = []
for word in list(csw_df['first_cs_word_zh']):
    doc = nlp(str(word))
    for token in doc:
        tags.append(token.tag_)
        break

In [9]:
coded_POS = []
for pos in tags:
    if pos == 'NN':
        coded_POS.append(0)
    elif pos == 'NR':
        coded_POS.append(0)
    elif pos == 'NT':
        coded_POS.append(0)
    elif pos == 'VE':
        coded_POS.append(1)
    elif pos == 'VV':
        coded_POS.append(1)
    else:
        coded_POS.append(2)

In [10]:
csw_df['first_cs_word_zh_pos'] = coded_POS
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,...,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos
0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",...,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1,2
1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],...,1,2,Y,you,2,3,october you mean finals,你,1,2
2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],...,1,2,Y,yah,0,3,can't tell whether we are yah,雅虎,2,0
3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",...,1,1,Y,but,2,3,"let me tell you one thing, but we can say that...",但,1,2
4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],...,1,1,Y,top,2,3,because top four,顶部,2,0


In [11]:
csw_df.to_csv('seame_with_eng_translations.csv')

In [35]:
csw_df = pd.read_csv('seame_with_eng_translations.csv')
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,...,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq
0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,...,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1,2,791724336,48791767
1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,...,Y,you,2,3,october you mean finals,你,1,2,950006304,25465651
2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,...,Y,yah,0,3,can't tell whether we are yah,雅虎,2,0,131496,11734
3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,...,Y,but,2,3,"let me tell you one thing, but we can say that...",但,1,2,791724336,48791767
4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,...,Y,top,2,3,because top four,顶部,2,0,40791068,0


In [53]:
first_cs_word_eng_freq = []
first_cs_word_zh_freq = []

total_count_eng = 356033418959
total_count_zh = 26859461025

first_cs_word_eng_raw_freq = list(csw_df['first_cs_word_eng_raw_freq'])
first_cs_word_zh_raw_freq = list(csw_df['first_cs_word_zh_raw_freq'])

for freq in first_cs_word_eng_raw_freq:
    if freq=='TODO':
        first_cs_word_eng_freq.append(freq)
    else:
        if int(freq)==0:
            first_cs_word_eng_freq.append(-math.log2(1/total_count_eng))
        else:
            first_cs_word_eng_freq.append(-math.log2(int(freq)/total_count_eng))
        

for freq in first_cs_word_zh_raw_freq:
    if freq=='TODO':
        first_cs_word_zh_freq.append(freq)
    else:
        if int(freq)==0:
            first_cs_word_zh_freq.append(-math.log2(1/total_count_zh))
        else:
            first_cs_word_zh_freq.append(-math.log2(int(freq)/total_count_zh))
        
csw_df['first_cs_word_eng_freq'] = first_cs_word_eng_freq
csw_df['first_cs_word_zh_freq'] = first_cs_word_zh_freq

csw_df.head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,...,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq
0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,...,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1,2,791724336,48791767,8.812799,9.104577
1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,...,2,3,october you mean finals,你,1,2,950006304,25465651,8.54986,10.042662
2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,...,0,3,can't tell whether we are yah,雅虎,2,0,131496,11734,21.368562,21.126304
3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,...,2,3,"let me tell you one thing, but we can say that...",但,1,2,791724336,48791767,8.812799,9.104577
4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,...,2,3,because top four,顶部,2,0,40791068,0,13.091472,34.644711
5,5,5,5,5,22,01NC02FBY_0101,Conversation,对 [呀] 那个 area five 多少,"['对', '那个', 'area', 'five', '多少']",5,...,0,4,how much is area five,区域,2,0,64951337,5054553,12.420366,12.375559
6,6,6,6,6,24,01NC02FBY_0101,Conversation,[诶] 那个 then 谁 拿 那个 top one 那个 foucia*,"['那个', 'then', '谁', '拿', '那个', 'top', 'one', '...",9,...,2,4,that then who takes that top one that foucia*,然后,2,2,283578912,5126199,10.294047,12.355253
7,7,7,7,7,31,01NC01FBX_0101,Conversation,因为 我 一直 bug 他 讲 [诶] result 几时 会 出 r...,"['因为', '我', '一直', 'bug', '他', '讲', 'result', '...",15,...,0,3,"because i have been bug, he said that result w...",漏洞,2,0,1082078,226962,18.327849,16.852620
8,8,8,8,8,35,01NC01FBX_0101,Conversation,就是 score 就是 score [mah] then 为什么 要 then...,"['就是', 'score', '就是', 'score', 'then', '为什么', ...",21,...,1,5,it is score that is the score then why he said...,分数,2,0,9348150,74680,15.216972,18.456277
9,9,9,9,9,36,01NC02FBY_0101,Conversation,[哎哟] 哪里 可能 那个 你 不 就 跟 我 讲 tricia 拿 ...,"['哪里', '可能', '那个', '你', '不', '就', '跟', '我', '讲...",13,...,0,6,where can you tell me about tricia and take on...,Tricia,6,0,127876,0,21.408836,34.644711


In [54]:
zh_translated_lines = []
for i in range(len(list(csw_df['filtered_transcript_2015']))):
    if i%100==0:
        print(i)
    ini_list = list(csw_df['filtered_transcript_2015'])[i]
    as_list = ini_list.strip('][').split(', ')
    as_list = [word[1:-1] for word in as_list if word.startswith("'") and word.endswith("'")]
    as_str = ' '.join(as_list)
    
    try:
        result = translator.translate(as_str, src='en', dest='zh-cn').text
        time.sleep(0.3)
        zh_translated_lines.append(result)
    except:
        zh_translated_lines.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [55]:
zh_translated_lines.count('TODO')

30

In [56]:
csw_df['chinese_translation'] = zh_translated_lines
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,...,word_length_eng,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation
0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,...,3,"uh, on october 11th, i tell you a thing to be ...",但,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去
1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,...,3,october you mean finals,你,1,2,950006304,25465651,8.54986,10.042662,十月你是说决赛
2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,...,3,can't tell whether we are yah,雅虎,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是
3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,...,3,"let me tell you one thing, but we can say that...",但,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...
4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,...,3,because top four,顶部,2,0,40791068,0,13.091472,34.644711,因为前四名


In [57]:
csw_df.to_csv('seame_with_eng_zh1_translations.csv')

In [3]:
csw_df = pd.read_csv('seame_with_eng_zh1_translations.csv')

In [28]:
def merge_spaces(list_of_strings):
    for i in range(len(list_of_strings)):
        try:
            if list_of_strings[i] == '':
                while i < len(list_of_strings)-1 and list_of_strings[i+1]=='':
                    list_of_strings.pop(i+1)
        except:
            pass
    return list_of_strings  

In [29]:
def merge_words(list_of_eng_strings):
    for i in range(len(list_of_eng_strings)):
        try:
            if list_of_eng_strings[i] != '':
                while i < len(list_of_eng_strings)-1 and list_of_eng_strings[i+1]!='':
                    list_of_eng_strings[i] = list_of_eng_strings[i] + ' ' + list_of_eng_strings[i+1]
                    list_of_eng_strings.pop(i+1)

        except:
            pass
    list_of_eng_strings = [word for word in list_of_eng_strings if word!='']
    return list_of_eng_strings

In [74]:
zh_trans_word_by_word = []
for i in range(len(list(csw_df['filtered_transcript_2015']))):
    if i%100==0:
        print(i)
        
    ini_list = list(csw_df['filtered_transcript_2015'])[i]
    as_list = ini_list.strip('][').split(', ')
    as_list = [word[1:-1] for word in as_list if word.startswith("'") and word.endswith("'")]
    as_str = ' '.join(as_list)

    eng_only = []
    zh_only = []
    for word in as_list:
        if word.isascii():
            eng_only.append(word)
            zh_only.append('')
        else:
            zh_only.append(word)
            eng_only.append('')
    
    zh_only = merge_spaces(zh_only)
    eng_only = merge_spaces(eng_only)
    
    try:
        for word in merge_words(eng_only):
            result = translator.translate(word, src='en', dest='zh-cn').text
            zh_only[zh_only.index('')] = result
        time.sleep(0.3)
        final_result = ''.join(zh_only)
        zh_trans_word_by_word.append(' '.join(jieba.lcut(final_result)))
        
    except:
        zh_trans_word_by_word.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [75]:
zh_trans_word_by_word.count('TODO')

926

In [76]:
csw_df['chinese_wbw_translation'] = zh_trans_word_by_word
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,...,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",...,"uh, on october 11th, i tell you a thing to be ...",但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",...,october you mean finals,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",...,can't tell whether we are yah,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",...,"let me tell you one thing, but we can say that...",但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",...,because top four,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名


In [77]:
csw_df.to_csv('seame_with_eng_zh2_translations.csv')

In [78]:
en_trans_word_by_word = []
for i in range(len(list(csw_df['filtered_transcript_2015']))):
    if i%100==0:
        print(i)
        
    ini_list = list(csw_df['filtered_transcript_2015'])[i]
    as_list = ini_list.strip('][').split(', ')
    as_list = [word[1:-1] for word in as_list if word.startswith("'") and word.endswith("'")]
    as_str = ' '.join(as_list)

    eng_only = []
    zh_only = []
    for word in as_list:
        if word.isascii():
            eng_only.append(word)
            zh_only.append('')
        else:
            zh_only.append(word)
            eng_only.append('')
    
    zh_only = merge_spaces(zh_only)
    eng_only = merge_spaces(eng_only)
    
    try:
        for word in merge_words(zh_only):
            result = translator.translate(word, src='zh-cn', dest='en').text
            eng_only[eng_only.index('')] = result
        time.sleep(0.3)
        final_result = ' '.join(eng_only)
        en_trans_word_by_word.append(final_result.lower())
        
    except:
        en_trans_word_by_word.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [79]:
en_trans_word_by_word.count('TODO')

4724

In [80]:
csw_df['english_wbw_translation'] = en_trans_word_by_word
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,...,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,english_wbw_translation
0,0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",...,但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,"uh, on october 11th, i tell you a thing to tel..."
1,1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",...,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,october you mean finals
2,2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",...,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,can't tell if we are yah
3,3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",...,但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,"i tell you one thing but can't tell us, we hav..."
4,4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",...,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名,because top four


In [81]:
csw_df.to_csv('seame_with_eng2_zh2_translations.csv')

In [115]:
csw_df = pd.read_csv('seame_with_eng2_zh2_translations.csv')

In [116]:
csw_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1',
       'file', 'type', 'transcript_2015', 'filtered_transcript_2015',
       'length_of_turn', 'start_of_code_switches',
       'length_of_code_switched_segments', 'direction_of_code_switches',
       'number_of_code_switches', 'intersentential_code_switch?',
       'type_of_code_switching', 'language', 'conversation', 'speaker',
       'marking', 'first_cs_word', 'first_cs_word_eng_pos', 'word_length_eng',
       'english_translation', 'first_cs_word_zh', 'word_length_zh',
       'first_cs_word_zh_pos', 'first_cs_word_eng_raw_freq',
       'first_cs_word_zh_raw_freq', 'first_cs_word_eng_freq',
       'first_cs_word_zh_freq', 'chinese_translation',
       'chinese_wbw_translation', 'english_wbw_translation'],
      dtype='object')

In [118]:
csw_df = csw_df[['file', 'type', 'conversation', 'speaker', 'filtered_transcript_2015', 'length_of_turn',
       'direction_of_code_switches', 'first_cs_word',
       'first_cs_word_eng_pos', 'word_length_eng', 'english_translation', 'english_wbw_translation', 'first_cs_word_zh', 'word_length_zh', 'first_cs_word_zh_pos',
       'first_cs_word_eng_raw_freq', 'first_cs_word_zh_raw_freq',
       'first_cs_word_eng_freq', 'first_cs_word_zh_freq',
       'chinese_translation', 'chinese_wbw_translation']]
csw_df.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名


In [119]:
without_todo = csw_df.replace('TODO', np.NaN)
without_todo = without_todo.dropna()
len(without_todo)

28333

In [120]:
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名


In [89]:
zh_df = df[df['language']=='ZH']
zh_df = zh_df[zh_df['length_of_turn']!=0]
zh_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker
2,01NC02FBY_0101,Conversation,我们,['我们'],1,-1,-1,-1,0,-1,-1,ZH,1,2
3,01NC02FBY_0101,Conversation,进 [啊] 没有,"['进', '没有']",2,-1,-1,-1,0,-1,-1,ZH,1,2
4,01NC02FBY_0101,Conversation,(ppl) 等一下 (ppb),"['等', '一下']",2,-1,-1,-1,0,-1,-1,ZH,1,2
9,01NC02FBY_0101,Conversation,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,-1,-1,-1,0,-1,-1,ZH,1,2
10,01NC02FBY_0101,Conversation,算,['算'],1,-1,-1,-1,0,-1,-1,ZH,1,2


In [121]:
zh_df.columns

Index(['file', 'type', 'transcript_2015', 'filtered_transcript_2015',
       'length_of_turn', 'start_of_code_switches',
       'length_of_code_switched_segments', 'direction_of_code_switches',
       'number_of_code_switches', 'intersentential_code_switch?',
       'type_of_code_switching', 'language', 'conversation', 'speaker'],
      dtype='object')

In [123]:
zh_df = zh_df[['file', 'type', 'conversation', 'speaker', 'transcript_2015', 'filtered_transcript_2015',
       'length_of_turn', 'language']]
zh_df.head()

,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language
2,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH
3,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH
4,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH
9,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH
10,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH


In [136]:
mean_lengths = []

for index, row in zh_df.iterrows():
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]

    length = 0
    for word in transcript:
        length += len(word)
    mean_lengths.append(length/len(transcript))

zh_df['mean word length'] = mean_lengths

In [147]:
random_length = []
random_word = []

for index, row in zh_df.iterrows():
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]

    word = random.choice(transcript)
    random_word.append(word)
    random_length.append(len(word))

zh_df['random word'] = random_word
zh_df['random word length'] = random_length

In [150]:
tags = []
for word in list(zh_df['random word']):
    doc = nlp(str(word))
    for token in doc:
        tags.append(token.tag_)
        break
        
coded_POS = []
for pos in tags:
    if pos == 'NN':
        coded_POS.append(0)
    elif pos == 'NR':
        coded_POS.append(0)
    elif pos == 'NT':
        coded_POS.append(0)
    elif pos == 'VE':
        coded_POS.append(1)
    elif pos == 'VV':
        coded_POS.append(1)
    else:
        coded_POS.append(2)
        
zh_df['random POS'] = coded_POS

In [160]:
mode_tags = []

for index, row in zh_df.iterrows():
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    
    tags = []
    for word in transcript:
        doc = nlp(str(word))
        for token in doc:
            tags.append(token.tag_)
            break
    coded_POS = []
    for pos in tags:
        if pos == 'NN':
            coded_POS.append(0)
        elif pos == 'NR':
            coded_POS.append(0)
        elif pos == 'NT':
            coded_POS.append(0)
        elif pos == 'VE':
            coded_POS.append(1)
        elif pos == 'VV':
            coded_POS.append(1)
        else:
            coded_POS.append(2)
    
    mode_tags.append(max(set(coded_POS), key=coded_POS.count))

In [161]:
zh_df['mode POS'] = mode_tags

In [171]:
total_count_zh = 26859461025
random_freqs = []

for i in range(len(list(zh_df['random word']))):
    if i%100 == 0:
        print(i)
    
    word = list(zh_df['random word'])[i]
    encoded_query = urllib.parse.quote(word)
    params = {'corpus': 'chi', 'query': encoded_query, 'topk': 1}
    params = '&'.join('{}={}'.format(name, value) for name, value in params.items())

    response = requests.get('https://api.phrasefinder.io/search?' + params)
    
    if response.status_code == 200:
        if response.json()['phrases'] == []:
            random_freqs.append(-math.log2(1/total_count_zh))
                    
        else:
            raw_freq = response.json()['phrases'][0]['mc']
            random_freqs.append(-math.log2(int(raw_freq)/total_count_zh))
    else:
        random_freqs.append('TODO')
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800


ConnectionError: HTTPSConnectionPool(host='api.phrasefinder.io', port=443): Max retries exceeded with url: /search?corpus=chi&query=%E4%BC%9A&topk=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbfe83f3490>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [172]:
len(random_freqs)

14862

In [175]:
for i in range(14862, len(list(zh_df['random word']))):
    if i%100 == 0:
        print(i)
    
    word = list(zh_df['random word'])[i]
    encoded_query = urllib.parse.quote(word)
    params = {'corpus': 'chi', 'query': encoded_query, 'topk': 1}
    params = '&'.join('{}={}'.format(name, value) for name, value in params.items())

    response = requests.get('https://api.phrasefinder.io/search?' + params)
    
    if response.status_code == 200:
        if response.json()['phrases'] == []:
            random_freqs.append(-math.log2(1/total_count_zh))
                    
        else:
            raw_freq = response.json()['phrases'][0]['mc']
            random_freqs.append(-math.log2(int(raw_freq)/total_count_zh))
    else:
        random_freqs.append('TODO')
    

14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900


ConnectionError: ('Connection aborted.', OSError(0, 'Error'))

In [176]:
len(random_freqs)

16922

In [177]:
for i in range(16922, len(list(zh_df['random word']))):
    if i%100 == 0:
        print(i)
    
    word = list(zh_df['random word'])[i]
    encoded_query = urllib.parse.quote(word)
    params = {'corpus': 'chi', 'query': encoded_query, 'topk': 1}
    params = '&'.join('{}={}'.format(name, value) for name, value in params.items())

    response = requests.get('https://api.phrasefinder.io/search?' + params)
    
    if response.status_code == 200:
        if response.json()['phrases'] == []:
            random_freqs.append(-math.log2(1/total_count_zh))
                    
        else:
            raw_freq = response.json()['phrases'][0]['mc']
            random_freqs.append(-math.log2(int(raw_freq)/total_count_zh))
    else:
        random_freqs.append('TODO')
    

17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
20100
20200
20300
20400
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700


In [178]:
print(random_freqs.count('TODO'))

3


In [179]:
zh_df['random word frequency'] = random_freqs

In [180]:
zh_df.to_csv('seame_non_cs_data.csv')

In [2]:
zh_df = pd.read_csv('seame_non_cs_data.csv')
zh_df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS,random word frequency,mean surprisal,random word surprisal
0,0,2,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2,8.519816876703524,24.420960,24.420960
1,1,3,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1,11.230889976662251,24.086453,24.284205
2,2,4,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1,13.208367276887378,23.196542,30.291875
3,3,9,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1,11.972945663039544,21.718952,24.101382
4,4,10,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1,13.708818971267016,24.420960,24.420960


In [21]:
# phrasefinder.io discontinued - now have to do manual calculations
# try searching googlebooks 1-grams first, then try 2-grams for frequencies not found in 1-grams

In [5]:
max = 0
for transcript in list(zh_df['filtered_transcript_2015']):
    transcript = transcript.strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    for word in transcript:
        if len(word)>max:
            max = len(word)
            print(word)
print(max)

我们
八十六
三天三夜
#beh-tahan#
#goreng-pisang#
#xiao-ding-dong#
16


In [6]:
num = False
for transcript in list(zh_df['filtered_transcript_2015']):
    transcript = transcript.strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    for word in transcript:
        num = any(char.isdigit() for char in word)

print(num)

False


In [8]:
google_books_df_0 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-0.gz', sep='\t', header=None)
google_books_df_0.head()

,0,1,2,3
0,"0,05_NOUN",1955,1,1
1,"0,05_NOUN",1957,1,1
2,"0,05_NOUN",1963,2,2
3,"0,05_NOUN",1966,1,1
4,"0,05_NOUN",1968,1,1


In [9]:
google_books_df_1 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-1.gz', sep='\t', header=None)
google_books_df_1.head()

,0,1,2,3
0,"1,116",1911,2,1
1,"1,116",1948,1,1
2,"1,116",1949,5,2
3,"1,116",1954,1,1
4,"1,116",1956,2,2


In [10]:
google_books_df = pd.concat([google_books_df_0, google_books_df_1], axis=0)

In [11]:
len(google_books_df) == len(google_books_df_0)+len(google_books_df_1)

True

In [12]:
len(google_books_df)

4268208

In [13]:
google_books_df_2 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-2.gz', sep='\t', header=None)

In [14]:
google_books_df = pd.concat([google_books_df, google_books_df_2], axis=0)
#len(google_books_df) == len(google_books_df_0)+len(google_books_df_1)+len(google_books_df_2)

In [15]:
google_books_df_3 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-3.gz', sep='\t', header=None)
google_books_df_4 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-4.gz', sep='\t', header=None)
google_books_df_5 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-5.gz', sep='\t', header=None)
google_books_df_6 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-6.gz', sep='\t', header=None)
google_books_df_7 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-7.gz', sep='\t', header=None)
google_books_df_8 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-8.gz', sep='\t', header=None)
google_books_df_9 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-9.gz', sep='\t', header=None)

In [16]:
google_books_df = pd.concat([google_books_df, google_books_df_3, google_books_df_4, google_books_df_5, google_books_df_6, google_books_df_7, google_books_df_8, google_books_df_9], axis=0)

In [17]:
len(google_books_df) == len(google_books_df_0)+len(google_books_df_1)+len(google_books_df_2)+len(google_books_df_3)+len(google_books_df_4)+len(google_books_df_5)+len(google_books_df_6)+len(google_books_df_7)+len(google_books_df_8)+len(google_books_df_9)

True

In [18]:
google_books_df_a = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-a.gz', sep='\t', header=None)
google_books_df_b = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-b.gz', sep='\t', header=None)
google_books_df_c = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-c.gz', sep='\t', header=None)
google_books_df_d = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-d.gz', sep='\t', header=None)
google_books_df_e = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-e.gz', sep='\t', header=None)
google_books_df_f = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-f.gz', sep='\t', header=None)
google_books_df_g = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-g.gz', sep='\t', header=None)
google_books_df_h = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-h.gz', sep='\t', header=None)
google_books_df_i = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-i.gz', sep='\t', header=None)
google_books_df_j = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-j.gz', sep='\t', header=None)
google_books_df_k = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-k.gz', sep='\t', header=None)
google_books_df_l = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-l.gz', sep='\t', header=None)
google_books_df_m = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-m.gz', sep='\t', header=None)
google_books_df_n = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-n.gz', sep='\t', header=None)
google_books_df_o = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-o.gz', sep='\t', header=None)
google_books_df_p = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-p.gz', sep='\t', header=None)
google_books_df_q = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-q.gz', sep='\t', header=None)
google_books_df_r = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-r.gz', sep='\t', header=None)
google_books_df_s = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-s.gz', sep='\t', header=None)
google_books_df_t = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-t.gz', sep='\t', header=None)
google_books_df_u = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-u.gz', sep='\t', header=None)
google_books_df_v = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-v.gz', sep='\t', header=None)
google_books_df_w = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-w.gz', sep='\t', header=None)
google_books_df_x = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-x.gz', sep='\t', header=None)
google_books_df_y = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-y.gz', sep='\t', header=None)
google_books_df_z = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-z.gz', sep='\t', header=None)
google_books_df_other = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-other.gz', sep='\t', header=None)
#google_books_df_pos = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-pos.gz', sep='\t', header=None)
#google_books_df_punctuation = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-punctuation.gz', sep='\t', header=None)

In [19]:
google_books_df = pd.concat([google_books_df, google_books_df_a, google_books_df_b, google_books_df_c, google_books_df_d, google_books_df_e, google_books_df_f, google_books_df_g, google_books_df_h, google_books_df_i, google_books_df_j, google_books_df_k, google_books_df_l, google_books_df_m, google_books_df_n, google_books_df_o, google_books_df_p, google_books_df_q, google_books_df_r, google_books_df_s, google_books_df_t, google_books_df_u, google_books_df_v, google_books_df_w, google_books_df_x, google_books_df_y, google_books_df_z, google_books_df_other], axis=0)

In [20]:
google_books_df = google_books_df.rename(columns={0: "ngram", 1: "year", 2: "match_count", 3: "volume_count"})

In [21]:
google_books_df.head()

,ngram,year,match_count,volume_count
0,"0,05_NOUN",1955,1,1
1,"0,05_NOUN",1957,1,1
2,"0,05_NOUN",1963,2,2
3,"0,05_NOUN",1966,1,1
4,"0,05_NOUN",1968,1,1


In [22]:
len(google_books_df)

24698298

In [23]:
cleaned_ngram = [str(word)[:(str(word).index('_'))] if '_' in str(word) else str(word) for word in list(google_books_df['ngram'])]
google_books_df['cleaned_ngram'] = cleaned_ngram

In [24]:
google_books_df.head()

,ngram,year,match_count,volume_count,cleaned_ngram
0,"0,05_NOUN",1955,1,1,"0,05"
1,"0,05_NOUN",1957,1,1,"0,05"
2,"0,05_NOUN",1963,2,2,"0,05"
3,"0,05_NOUN",1966,1,1,"0,05"
4,"0,05_NOUN",1968,1,1,"0,05"


In [25]:
google_books_df.tail()

,ngram,year,match_count,volume_count,cleaned_ngram
1096,Ⅲ,2003,5,4,Ⅲ
1097,Ⅲ,2005,17,16,Ⅲ
1098,Ⅲ,2006,10,6,Ⅲ
1099,Ⅲ,2007,12,4,Ⅲ
1100,Ⅲ,2008,1,1,Ⅲ


In [26]:
match_count_int = [int(num) for num in list(google_books_df['match_count'])]
google_books_df['match_count_as_int'] = match_count_int
google_books_df.head()

,ngram,year,match_count,volume_count,cleaned_ngram,match_count_as_int
0,"0,05_NOUN",1955,1,1,"0,05",1
1,"0,05_NOUN",1957,1,1,"0,05",1
2,"0,05_NOUN",1963,2,2,"0,05",2
3,"0,05_NOUN",1966,1,1,"0,05",1
4,"0,05_NOUN",1968,1,1,"0,05",1


In [ ]:
total_count_zh = 26859461025
unique_seame_words = defaultdict()
for index, row in zh_df.iterrows():
    #print(str(index/22774*100)+'% done')
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    for word in transcript:
        if len(word)>2: #doing this to stick to using 1-gram data (bigram data is too large)
            word = word[:2]
        unique_seame_words[word] = 1/total_count_zh

In [ ]:
len(list(unique_seame_words.keys()))

In [ ]:
for i in range(len(list(unique_seame_words.keys()))):
    print(str(i/len(list(unique_seame_words.keys()))*100)+'% done')
    word = list(unique_seame_words.keys())[i]
    sub_df = google_books_df[google_books_df['cleaned_ngram']==word]
    word_freq_total = sub_df['match_count_as_int'].sum()
    if word_freq_total != 0:
        unique_seame_words[word] = word_freq_total/total_count_zh
    

In [ ]:
unique_seame_words

In [ ]:
mean_freqs = []

for index, row in zh_df.iterrows():
    print(str(index/22774*100)+'% done')
    
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    
    freqs = []
    for word in transcript:
        if len(word)>2: #doing this to stick to using 1-gram data (bigram data is too large)
            word = word[:2]
        freqs.append(unique_seame_words[word])
        
    try:
        mean_freqs.append(-math.log2(np.mean(freqs)))
    except:
        mean_freqs.append('TODO')

print(mean_freqs.count('TODO'))
        
zh_df['mean word frequency'] = mean_freqs

In [ ]:
len(mean_freqs)

In [ ]:
zh_df.to_csv('seame_non_cs_data_mean_freqs.csv') #rename this file if all looks good
zh_df.head()

In [ ]:
zh_df.to_csv('seame_non_cs_data.csv')

In [6]:
transcripts_as_lst = list(zh_df['filtered_transcript_2015'])
cleaned_transcripts = []
for transcript in transcripts_as_lst:
    cleaned_transcript = transcript.strip('][').split(', ')
    cleaned_transcript = [word[1:-1] for word in cleaned_transcript if word.startswith("'") and word.endswith("'")]
    cleaned_transcripts.append(cleaned_transcript)
cleaned_transcripts
transcripts_as_str = [' '.join(lst) for lst in cleaned_transcripts]
transcripts_as_str

['我们',
 '进 没有',
 '等 一下',
 '我们 拿 多少 分 到 最后',
 '算',
 '一百',
 '八十六 加 八十 加',
 '一 六',
 '八十六 加',
 '没有 没有 八 八十 没有 算',
 '我 拿 一 五 八',
 '就 八十 我 拿 了 一 三 九',
 '一 三 九 八十六',
 '一八 四',
 '没有 我 拿 比较 高 一点点 一 五 八',
 '一 七八',
 '你 拿 多少 一 五一',
 '还是 我 是',
 '十几 号 讲 多 一次 十一月 十号',
 '十月 十一号',
 '五 六七',
 '我们',
 '十 号 十月 对 不 对',
 '十号 十月',
 '十月 十号 听 起来 像是 很 没有 空 的 时候',
 '对 拜 六',
 '很 早 很 早 我 忘记 了 去年 几时',
 '就 给 我们 射 先',
 '这样',
 '会 有 失手 的 时候',
 '赢 我们 二十多',
 '什么 意思',
 '他们 才 五六 多',
 '对 他们 的 五六 多 是',
 '这样 而已 我 还 以为 是 那种 过 六头 的',
 '去年 你们 你 的 那个 一七零 加 他们 的 一 就 过 了 六百 了',
 '对 对 对 就 连 做 三天三夜',
 '我们 呃 我 我 讲 若惠',
 '他 就 讲',
 '他 就 讲',
 '差不多 若惠 她 讲 她 六点 多',
 '我们',
 '我 没有 驾车',
 '是 你 跟 我 讲 他 有 驾车',
 '八点',
 '我 就 看到',
 '庆辉 就 问',
 '他 就 会',
 '他 就 碎碎念 碎碎念',
 '她 就 讲',
 '呃',
 '他 就 讲',
 '诶',
 '呃 对 对',
 '对 对 对',
 '讲 了 什么 东西',
 '我 忘记 了 了',
 '对',
 '我 就 #beh-tahan# 他',
 '刚才 瑞宝 跟 我 讲',
 '对',
 '对 对',
 '那个 叫 什么 名',
 '那个 叫 什么 名 我们 那个 对',
 '不 认识',
 '因为 他 每次 呃',
 '呃 读 不 完',
 '就 本来 是',
 '没有 没有 他 有',
 '屁',
 '我 没有 什么 打算 要 做',
 '忘记 了',
 '他们',
 '屁 没有 打包 他 没有 

In [7]:
file = open('non_cs_transcripts.txt','w')
for line in transcripts_as_str:
    file.write(line +"\n")
file.close()

In [ ]:
#what to do about no cs points in non-cs sentences? just choose a random word?

#do all again using other training data for models?
#need to address conll cs1 reviews --> see google doc for additional analyses required

# NOTE: translations in both directions are pretty bad on this data set. 

In [91]:
en_df = df[df['language']=='EN']
en_df = en_df[en_df['length_of_turn']!=0]
en_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker
6,01NC01FBX_0101,Conversation,[ah] final saturday,"['final', 'saturday']",2,-1,-1,-1,0,-1,-1,EN,1,1
26,01NC01FBX_0101,Conversation,area five,"['area', 'five']",2,-1,-1,-1,0,-1,-1,EN,1,1
38,01NC02FBY_0101,Conversation,ya [lor],['ya'],1,-1,-1,-1,0,-1,-1,EN,1,2
40,01NC02FBY_0101,Conversation,[oh] my god,"['my', 'god']",2,-1,-1,-1,0,-1,-1,EN,1,2
57,01NC02FBY_0101,Conversation,not very so wish,"['not', 'very', 'so', 'wish']",4,-1,-1,-1,0,-1,-1,EN,1,2


In [92]:
len(en_df)

22529